In [ ]:

from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
import matplotlib.pyplot as plt
from sklearn import metrics 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler


import pandas as pd
import numpy as np
np.random.seed(42)
import seaborn as sns

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib.gridspec import GridSpec


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
dfchallenger = pd.read_pickle('complete_details_matches_challenger.pkl')

In [ ]:
dfgrandmaster = pd.read_pickle('complete_details_matches_grandmaster.pkl')

In [ ]:
dfmaster = pd.read_pickle('details_matches_master_complete.pkl')

In [ ]:
columns = ["gameId","teamAWins","teamAWardsPlaced","teamAWardsDestroyed","teamAFirstBlood","teamAKills","teamADeaths","teamAAssists","teamAEliteMonsters","teamABaron","teamADragons","teamAHeralds","teamATowersDestroyed","teamATotalGold","teamAAvgLevel","teamATotalExperience","teamATotalMinionsKilled","teamAGoldDiff","teamAExperienceDiff","teamAGoldPerMin","teamBWardsPlaced","teamBWardsDestroyed","teamBFirstBlood","teamBKills","teamBDeaths","teamBAssists","teamBEliteMonsters","teamABaron","teamBDragons","teamBHeralds","teamBTowersDestroyed","teamBTotalGold","teamBAvgLevel","teamBTotalExperience","teamBTotalMinionsKilled","teamBGoldDiff","teamBExperienceDiff","teamBGoldPerMin"]
dict_matches = {}

In [ ]:
dfTotal = dfchallenger.copy()
dfTotal = dfTotal.append(dfgrandmaster.copy())
dfTotal = dfTotal.append(dfmaster.copy())
len(dfTotal)

In [ ]:
dict_picks = {}
dict_bans = {}

In [ ]:
#row['info.teams'][0]['bans']
champss = {}
dict_bans = {}
for index, row in dfTotal.iterrows():
    gameid = row['metadata.matchId']
    dict_bans[gameid] = {}
    dict_bans[gameid]['gameId'] = gameid
    for n in row['info.teams']:
        #print(n['bans'])
        if (n['teamId'] == 100):
            team = "teamA"
            teamAWins = int(n['win'])
        elif (n['teamId'] == 200):
            team = "teamB"
        for b in n['bans']:
            turn = b['pickTurn']
            champ = b['championId']
            if turn > 5:
                turn -= 5

            if b['championId'] == -1:
                champ = 0
            dict_bans[gameid][team + "Ban" + str(turn)] = champ
    dict_bans[gameid]['teamAWins'] = teamAWins
    
pd.set_option('display.max_columns', 40)
dbans = pd.DataFrame(dict_bans)
dbans_T = dbans.T
dbans_T = dbans_T.reset_index(drop=True)
dbans_T = dbans_T.convert_dtypes()
dbans_T.to_csv("planilha_bans_final.csv", index=False)
dbans_T

In [ ]:
champss = {}
dict_picks = {}
for index, row in dfTotal.iterrows():
    gameid = row['metadata.matchId']
    dict_picks[gameid] = {}
    dict_picks[gameid]['gameId'] = gameid
    for n in row['info.participants']:
        if (n['teamId'] == 100):
            team = "teamA"
            teamAWins = int(n['win'])
        elif (n['teamId'] == 200):
            team = "teamB"
        turn = n['participantId']
        champ = n['championId']
        champss[champ] = 1

        if turn > 5:
            turn -= 5

        if b['championId'] == -1:
            champ = 0
        dict_picks[gameid][team + "Pick" + str(turn)] = champ
    dict_picks[gameid]['teamAWins'] = teamAWins
    
pd.set_option('display.max_columns', 40)
dpicks = pd.DataFrame(dict_picks)
dpicks_T = dpicks.T
dpicks_T = dpicks_T.reset_index(drop=True)
#dm_T = dm_T.reindex(sorted(columns), axis=1)
dpicks_T = dpicks_T.convert_dtypes()
dpicks_T.to_csv("planilha_picks_final.csv", index=False)
dpicks_T

In [ ]:
turns = [i for i in range(1,6)]
turns

In [ ]:
dict_matches = {}
for index, row in dfTotal.iterrows():
    match = {}
    #print(index)
    #print(row['metadata.matchId'])
    gameid = row['metadata.matchId']
    gameid = int(gameid.replace("BR1_", ""))
    match[gameid] = {}
    match[gameid]['gameId'] = gameid
    for n in row['info.teams']:
        obj = n['objectives']
        #print(obj)
        if (n['teamId'] == 100):
            team = "teamA"
            #teamATurrestLost = obj['tower']['kills']
            #wardPlaced = int(n['wardsPlaced'])
            match[gameid]['teamAWins'] = int(n['win'])
        elif (n['teamId'] == 200):
            team = "teamB"
            #wardPlaced = n['wardsPlaced']

        #print(team)
        barons = int(obj['riftHerald']['kills'])
        if barons > 0:
            barons = int(1)
        match[gameid][team + 'Baron'] = barons
        
        firstBlood = int(obj['champion']['first'])
        match[gameid][team + 'FirstBlood'] = firstBlood

        dragons = int(obj['dragon']['kills'])
        #print(dragons)
        match[gameid][team + 'Dragons'] = dragons

        heralds = int(obj['riftHerald']['kills'])
        if heralds > 0:
            heralds = int(1)
        match[gameid][team + 'Heralds'] = heralds
            
        towersDestroyed = int(obj['tower']['kills'])
        match[gameid][team + 'TowersDestroyed'] = towersDestroyed

    match[gameid]['teamAEliteMonsters'] = match[gameid]['teamABaron'] + match[gameid]['teamAHeralds']
    match[gameid]['teamBEliteMonsters'] = match[gameid]['teamBHeralds'] + match[gameid]['teamBBaron']


    if match[gameid]['teamBDragons'] > match[gameid]['teamADragons']:
        #print("BD >> AD")
        match[gameid]['teamBDragons'] = int(1)
        match[gameid]['teamADragons'] = int(0)
    elif match[gameid]['teamBDragons'] < match[gameid]['teamADragons']:
        #print("BD << AD")
        match[gameid]['teamBDragons'] = int(0)
        match[gameid]['teamADragons'] = int(1)
    else:
        #print("BD == AD")
        match[gameid]['teamBDragons'] = int(0)
        match[gameid]['teamADragons'] = int(0)

    for n in row['info.participants']:
        team = ""
        role = ""
        if (n['teamId'] == 100):
            team = "teamA"
            match[gameid]['teamAWins'] = int(n['win'])
        elif (n['teamId'] == 200):
            team = "teamB"

        if n['individualPosition'] == "TOP":
            role = "TOP"
        elif n['individualPosition'] == "JUNGLE":
            role = "JG"
        elif n['individualPosition'] == "MIDDLE":
            role = "MID"
        elif n['individualPosition'] == "BOTTOM":
            role = "ADC"
        elif n['individualPosition'] == "UTILITY":
            role = "SUP"

        turn = n['participantId']
        champ = n['championId']

        if turn > 5:
            turn -= 5

        if champ == -1:
            champ = 0


        wardPlaced = int(n['wardsPlaced'])
        #print(wardPlaced)
        wardsKilled = int(n['wardsKilled'])
        #print(wardsKilled)
        kills = int(n['kills'])
        #print(kills)
        deaths = int(n['deaths'])
        #print(deaths)
        assists = int(n['assists'])
        #print(assists)
        goldEarned = int(n['goldEarned'])
        #print(goldEarned)
        AvgLevel = float(n['champLevel'])
        #print(AvgLevel)
        TotalExperience = int(n['champExperience'])
        #print(TotalExperience)
        TotalMinionsKilled = int(n['totalMinionsKilled'])
        #print(TotalMinionsKilled)
        GoldPerMin = float(n['challenges']['goldPerMinute'])
        #print(GoldPerMin)

        kda = 2*kills+assists-3*deaths
        match[gameid][team + 'Kda' + role] = kda
        match[gameid][team + "Pick" + role] = champ

        try:
            match[gameid][team + 'WardsPlaced'] += wardPlaced
        except:
            match[gameid][team + 'WardsPlaced'] = wardPlaced

        try:
            match[gameid][team + 'WardsDestroyed'] += wardsKilled
        except:
            match[gameid][team + 'WardsDestroyed'] = wardsKilled

        try:
            match[gameid][team + 'Kills'] += kills
        except:
            match[gameid][team + 'Kills'] = kills

        try:
            match[gameid][team + 'Deaths'] += deaths
        except:
            match[gameid][team + 'Deaths'] = deaths

        try:
            match[gameid][team + 'Assists'] += assists
        except:
            match[gameid][team + 'Assists'] = assists

        try:
            match[gameid][team + 'TotalGold'] += goldEarned
        except:
            match[gameid][team + 'TotalGold'] = goldEarned

        try:
            match[gameid][team + 'AvgLevel'] += AvgLevel
        except:
            match[gameid][team + 'AvgLevel'] = AvgLevel

        try:
            match[gameid][team + 'TotalExperience'] += TotalExperience
        except:
            match[gameid][team + 'TotalExperience'] = TotalExperience

        try:
            match[gameid][team + 'TotalMinionsKilled'] += TotalMinionsKilled
        except:
            match[gameid][team + 'TotalMinionsKilled'] = TotalMinionsKilled
        
        try:
            match[gameid][team + 'GoldPerMin'] += GoldPerMin
        except:
            match[gameid][team + 'GoldPerMin'] = GoldPerMin

    match[gameid]['teamAExperienceDiff'] = 0
    match[gameid]['teamBExperienceDiff'] = 0
    match[gameid]['teamAExperienceDiff'] = match[gameid]['teamATotalExperience'] - match[gameid]['teamBTotalExperience']
    match[gameid]['teamBExperienceDiff'] = match[gameid]['teamBTotalExperience'] - match[gameid]['teamATotalExperience']
    match[gameid]['teamBAvgLevel'] = float(match[gameid]['teamBAvgLevel'])/float(5.0)
    match[gameid]['teamAAvgLevel'] = float(match[gameid]['teamAAvgLevel'])/float(5.0)

    match[gameid]['teamAGoldDiff'] = 0
    match[gameid]['teamBGoldDiff'] = 0
    try:
        match[gameid]['teamAGoldDiff'] = match[gameid]['teamATotalGold'] - match[gameid]['teamBTotalGold']
    except:
        del match[gameid]
        continue
    try:
        match[gameid]['teamBGoldDiff'] = match[gameid]['teamBTotalGold'] - match[gameid]['teamATotalGold']
    except:
        del match[gameid]
        continue
    
    dict_matches[gameid] = {}
    dict_matches[gameid]['gameId'] = int(gameid)

    turns = [i for i in range(1,6)]
    roles = ["TOP", "JG", "MID", "ADC", "SUP"]
    teams = ["teamA", "teamB"]
    #print()
    #print("####")
    try:
        dict_matches[gameid]['teamAWins'] = int(match[gameid]['teamAWins'] )
        for team in teams:
            dict_matches[gameid][team + 'WardsPlaced'] = int(match[gameid][team + 'WardsPlaced'])
            dict_matches[gameid][team + 'WardsDestroyed'] = int(match[gameid][team + 'WardsDestroyed'])
            dict_matches[gameid][team + 'FirstBlood'] = int(match[gameid][team + 'FirstBlood'] )
            dict_matches[gameid][team + 'Kills'] = int(match[gameid][team + 'Kills'])
            dict_matches[gameid][team + 'Deaths'] = int(match[gameid][team + 'Deaths'])
            dict_matches[gameid][team + 'Assists'] = int(match[gameid][team + 'Assists'])
            for role in roles:
                dict_matches[gameid][team + "Pick" + role] = int(match[gameid][team + "Pick" + role])
                dict_matches[gameid][team + 'Kda' + role] = float(match[gameid][team + 'Kda' + role])

            dict_matches[gameid][team + 'EliteMonsters'] = int(match[gameid][team + 'EliteMonsters'])
            dict_matches[gameid][team + 'Dragons'] = int(match[gameid][team + 'Dragons'])
            dict_matches[gameid][team + 'Heralds'] = int(match[gameid][team + 'Heralds'])
            dict_matches[gameid][team + 'Baron'] = int(match[gameid][team + 'Baron'])
            dict_matches[gameid][team + 'TowersDestroyed'] = int(match[gameid][team + 'TowersDestroyed'] )
            dict_matches[gameid][team + 'TotalGold'] = int(match[gameid][team + 'TotalGold'])
            dict_matches[gameid][team + 'AvgLevel'] = float(match[gameid][team + 'AvgLevel'])
            dict_matches[gameid][team + 'TotalExperience'] = int(match[gameid][team + 'TotalExperience'])
            dict_matches[gameid][team + 'TotalMinionsKilled'] = int(match[gameid][team + 'TotalMinionsKilled'])
            dict_matches[gameid][team + 'GoldDiff'] = int(match[gameid][team + 'GoldDiff'])
            dict_matches[gameid][team + 'ExperienceDiff'] = int(match[gameid][team + 'ExperienceDiff'])
            dict_matches[gameid][team + 'GoldPerMin'] = float(match[gameid][team + 'GoldPerMin'])
    except Exception as error:
        print(error)
        del match[gameid]
        del dict_matches[gameid]
        continue

dict_matches

In [ ]:
pd.set_option('display.max_columns', 58)
dm = pd.DataFrame(dict_matches)
dm_T = dm.T
dm_T = dm_T.reset_index(drop=True)
dm_T = dm_T.convert_dtypes()
dm_T.to_pickle("planilha_total_sem_filtro.pkl")
dm_T

In [ ]:
gameidRemove = {}
dm_T_copy = dm_T.copy()
arr = dm_T.keys().values
keys = ['teamAFirstBlood','gameId', 'teamAWins', 'teamAFirstBlood', 'teamAPickTOP', 'teamAPickJG',  'teamAPickMID','teamAPickADC', 'teamAPickSUP', 
'teamBWins',  'teamBPickTOP', 'teamBPickJG',  'teamBPickMID','teamBPickADC', 'teamBPickSUP', 'teamBFirstBlood' ]
for k in keys: 
    index = np.where(arr == k)
    arr = np.delete(arr, index)
arr
for i in arr:
    print(i)
    arr1 = dm_T[i].values

    # primeiro quartil
    q1 = np.quantile(arr1, 0.25)
    
    # terceiro quartil
    q3 = np.quantile(arr1, 0.75)
    med = np.median(arr1)
    
    # ponto medio
    iqr = q3-q1
    
    # limites superiores e inferiores
    factor = 1.5
    upper_bound = q3+(factor*iqr)
    lower_bound = q1-(factor*iqr)
    print(lower_bound, iqr, upper_bound )

    countMax = 0
    countMin = 0
    for index, row in dm_T.iterrows():
        willRemove = row[i]
        if willRemove > upper_bound or willRemove < lower_bound:
            if willRemove > upper_bound:
                gameidRemove[row['gameId']] = 1
            elif willRemove < lower_bound:
                gameidRemove[row['gameId']] = 1


gameidRemove, len(gameidRemove)

In [ ]:
dm_T_copy = dm_T.copy()
print(len(gameidRemove))
for j, id in enumerate(gameidRemove):
    print(j)
    if id in dm_T_copy['gameId'].values:
        i = dm_T_copy[((dm_T_copy.gameId == id))].index
        dm_T_copy = dm_T_copy.drop(i).copy()
        print("removed")
    else:
        print("not removed")
dm_T_copy = dm_T_copy.reset_index(drop=True)
dm_T_copy.to_csv("planilha_total_com_filtro.csv", index=False)
dm_T_copy.to_pickle("planilha_total_com_filtro.pkl")
dm_T_copy

In [ ]:
df = pd.read_csv('planilha_total_com_filtro.csv')
df

In [ ]:
pd.set_option('display.max_columns', 8)

df.head()

In [ ]:
y = df['teamAWins'].copy()
X = df.drop(['gameId', 'teamAWins'], axis=1)
X

In [ ]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
print(X_sel_train.shape, X_sel_test.shape, y_sel_train.shape, y_sel_test.shape)

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test, digits=4, 
                   figsize=(10,5), params=False): 
        
    # Get Predictions
    y_hat_test = model.predict(X_test)
    y_hat_train = model.predict(X_train)
    
    # Classification Report / Scores 

    print("****CLASSIFICATION REPORT - TRAINING DATA****")
    
    print(metrics.classification_report(y_train,y_hat_train, digits=digits))
    

    print("****CLASSIFICATION REPORT - TEST DATA****")
    
    print(metrics.classification_report(y_test,y_hat_test, digits=digits))

    print("****CONFUSION MATRIX AND ROC-AUC VISUALIZATION****")

    fig, axes = plt.subplots(ncols=2, 
                             figsize=figsize)
    
    # Confusion Matrix
    metrics.plot_confusion_matrix(model, X_test,
                                  y_test,
                                 cmap='Purples',ax=axes[0])
    axes[0].set_title('Confusion Matrix')
    
    # Plot ROC Curve
    metrics.plot_roc_curve(model,X_test,y_test,ax=axes[1])

    ax = axes[1]
    ax.legend()
    ax.plot([0,1],[0,1], ls='-')
    ax.grid()
    ax.set_title('ROC AUC Curve')
    
    plt.tight_layout()
    plt.show()
    
    if params == True:
        print("****MODEL PARAMETERS****")
        params = pd.DataFrame(pd.Series(model.get_params()))
        params.columns=['parameters']
        display(params)

In [ ]:
dec_tree = DecisionTreeClassifier()

params = {'criterion':['gini', 'entropy','log_loss'],
        'max_depth': [8,13,21,34, 55, 89, 144, 233, 1000],
        'min_samples_split': [2, 3, 4], 'max_leaf_nodes': list(range(2, 100))}

dec_tree_grid = GridSearchCV(dec_tree, params, verbose=1, cv=3)
dec_tree_grid.fit(X_train, y_train)

In [ ]:
dec_tree_grid.best_params_, dec_tree_grid.best_score_

In [ ]:
evaluate_model(dec_tree_grid.best_estimator_, X_train, y_train, \
               X_test, y_test, params=True)

In [ ]:
kn = KNeighborsClassifier()

params = {'n_neighbors':[8,13, 21, 34, 55],
        'algorithm':['kd_tree', 'brute'],
        'weights':['distance', 'uniform'],
        'metric':['manhattan', '“euclidean', 'chebyshev']}

kn_grid = GridSearchCV(kn, params, verbose=1, cv=3)
kn_grid.fit(X_train, y_train)

In [ ]:
kn_grid.best_params_, kn_grid.best_score_

In [ ]:
evaluate_model(kn_grid.best_estimator_, X_train, y_train, \
               X_test, y_test, params=True)

In [ ]:
params = {'gamma':['auto', 1, 10, 100, 1e-2],
        'kernel':['linear','poly', 'rbf', 'sigmoid'],
        "C": [1, 10, 100, 1000]}

svc = SVC()

svc_grid = GridSearchCV(svc, params, verbose=1, cv=3)
svc_grid.fit(X_train, y_train)

In [ ]:
svc_grid.best_params_, svc_grid.best_score_

In [ ]:
evaluate_model(svc_grid.best_estimator_, X_train, y_train, \
               X_test, y_test, params=True)